# **VERSA BOT**

Description:
**VersaBot** is a versatile and powerful chatbot designed to handle a wide range of interactions with cutting-edge technology. It excels in:

Fast Conversational AI: Utilizing Groq’s high-speed model for rapid and smooth conversational exchanges, ensuring efficient and responsive interactions.
Advanced Document Handling: Process and analyze uploaded PDFs using RAG techniques, allowing users to query document content effectively.
Voice and Audio Interaction: Convert text to speech and process voice inputs, enabling interactive voice-based conversations and responses.
Multimedia Processing: Extract and transcribe text from both audio and video files, enabling users to interact based on multimedia content.
YouTube Content Integration: Transcribe and query content from YouTube videos, facilitating easy access to information from video platforms.
Language Translation: Seamlessly translate text between languages using advanced translation tools, catering to multilingual needs.
Machine Learning Guidance: Offers support and guidance for machine learning projects through the CrewAI Machine Learning Assistant integration.
VersaBot is your go-to assistant for intelligent conversations, document processing, multimedia content analysis, and multilingual support, making it an invaluable tool for diverse and dynamic interactions.

# LIBRARIES

In [1]:
!pip install langchain langchain-groq




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 965.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 858.4 kB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


# **CHAT WITH GROQ**

This Colab interface enables users to interact with a chatbot by selecting from models like mixtral-8x7b-32768, llama2-70b-4096, or gemma-7b-it, adjusting memory length, and submitting questions. It also includes functionality to clear inputs and view responses, leveraging the ChatGroq API for AI-driven conversations and memory management with ConversationBufferWindowMemory.

In [5]:
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')

import ipywidgets as widgets
from IPython.display import display, clear_output
from langchain_groq import ChatGroq
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# Define widgets
model_dropdown = widgets.Dropdown(
    options=['mixtral-8x7b-32768', 'llama2-70b-4096', 'gemma-7b-it'],
    value='mixtral-8x7b-32768',
    description='Choose a model:',
    style={'description_width': 'initial'}
)

memory_slider = widgets.IntSlider(
    value=50,
    min=50,
    max=800,
    step=1,
    description='Memory length:',
    style={'description_width': 'initial'},
    continuous_update=False
)

question_text = widgets.Textarea(
    value='',
    placeholder='Ask a question...',
    description='Question:',
    layout=widgets.Layout(width='100%', height='150px'),
    style={'description_width': 'initial'}
)

output_area = widgets.Output()

def handle_submit(button):
    with output_area:
        clear_output()
        model = model_dropdown.value
        conversational_memory_length = memory_slider.value
        user_question = question_text.value

        # Initialize memory and Groq chat object
        memory = ConversationBufferWindowMemory(k=conversational_memory_length)
        groq_chat = ChatGroq(
            groq_api_key=groq_api_key,
            model_name=model
        )
        conversation = ConversationChain(
            llm=groq_chat,
            memory=memory
        )

        # Process user question
        if user_question:
            response = conversation(user_question)
            print(f"**Chatbot Response:** {response['response']}")

def handle_clear(button):
    model_dropdown.value = 'mixtral-8x7b-32768'
    memory_slider.value = 50
    question_text.value = ''
    with output_area:
        clear_output()
        print("Inputs cleared.")

# Create and display buttons
submit_button = widgets.Button(description='Submit')
submit_button.on_click(handle_submit)

clear_button = widgets.Button(description='Clear')
clear_button.on_click(handle_clear)

# Display the widgets
display(
    widgets.VBox([
        model_dropdown,
        memory_slider,
        question_text,
        widgets.HBox([submit_button, clear_button]),
        output_area
    ])
)


# **CHAT WITH DOC**

**libraries**

In [7]:
!pip install langchain langchain-community faiss-cpu huggingface-hub pdfminer.six nltk rouge-score ipywidgets
!pip install unstructured[all]
!pip  install unstructured_inference
!pip install pi_heif
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr
!pip install pytesseract
!pip install unstructured
!pip install langchain_community langchain_text_splitters langchain_groq
!pip install sentence-transformers


  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 67.9 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.1 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7bec291e036f2ab9eb9d17eb09c22dc446df2e884140748f495d220a5ef46bc3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (301 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
T

# **CHAT WITH DOC**

This script processes PDF documents to extract text and set up a vector store using FAISS for semantic search. It utilizes a Retrieval-Augmented Generation (RAG) approach, where the vector store indexes text chunks, and a language model (via ChatGroq) generates responses based on user queries. The ConversationalRetrievalChain integrates the retrieval and generation components to handle user interactions effectively.

In [10]:
import os
import time
import tempfile
import pandas as pd
import pdfplumber
import ipywidgets as widgets
from IPython.display import display
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# Function to load PDF document using pdfplumber
def load_document_with_pdfplumber(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            documents = [{'text': page.extract_text()} for page in pdf.pages if page.extract_text()]
        return documents
    except Exception as e:
        print(f"Error loading document: {e}")
        return []

# Function to set up vectorstore
def setup_vectorstore(documents):
    try:
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200)
        texts = [doc['text'] for doc in documents]
        doc_chunks = text_splitter.create_documents(texts)
        return FAISS.from_documents(doc_chunks, embeddings)
    except Exception as e:
        print(f"Error setting up vectorstore: {e}")
        return None

# Function to create conversational chain
def create_chain(vectorstore, model_name):
    try:
        llm = ChatGroq(
            groq_api_key=groq_api_key,
            model_name=model_name,
            temperature=0
        )
        retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
        memory = ConversationBufferMemory(
            llm=llm,
            output_key="answer",
            memory_key="chat_history",
            return_messages=True
        )
        return ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=retriever,
            chain_type="map_reduce",
            memory=memory
        )
    except Exception as e:
        print(f"Error creating chain: {e}")
        return None

# Create widgets for the interface
upload_widget = widgets.FileUpload(accept='.pdf', multiple=False)
model_dropdown = widgets.Dropdown(
    options=[
        ('Mixtral 8x7b 32768', 'mixtral-8x7b-32768'),
        ('LLAMA 2 70b 4096', 'llama2-70b-4096'),
        ('Gemma 7b IT', 'gemma-7b-it'),
    ],
    value='mixtral-8x7b-32768',
    description='Model:',
)
process_button = widgets.Button(description="Process PDF", button_style='success')
text_input = widgets.Text(placeholder="Ask your question...", description="Your Question:")
query_button = widgets.Button(description="Get Response", button_style='primary')
output_area = widgets.Output()

# Display widgets in a structured layout
input_widgets = widgets.VBox([
    widgets.HBox([widgets.Label("Upload PDF:"), upload_widget]),
    widgets.HBox([widgets.Label("Select Model:"), model_dropdown]),
    process_button
])

interaction_widgets = widgets.VBox([
    text_input,
    query_button,
    output_area
])

display(input_widgets, interaction_widgets)

# Function to handle PDF processing
def on_process_button_clicked(b):
    with output_area:
        output_area.clear_output()
        if upload_widget.value:
            try:
                uploaded_file = list(upload_widget.value.values())[0]
                file_content = uploaded_file['content']
                file_path = tempfile.mktemp(suffix='.pdf')
                with open(file_path, "wb") as f:
                    f.write(file_content)
                documents = load_document_with_pdfplumber(file_path)
                if not documents:
                    print("No text found in the PDF.")
                    return
                vectorstore = setup_vectorstore(documents)
                if vectorstore is None:
                    print("Failed to create vectorstore.")
                    return
                model_name = model_dropdown.value
                global chain_instance
                chain_instance = create_chain(vectorstore, model_name)
                print("PDF processed and model ready.")
            except Exception as e:
                print(f"An error occurred: {e}")
        else:
            print("Please upload a PDF file.")

process_button.on_click(on_process_button_clicked)

# Function to handle query processing
def on_query_button_clicked(b):
    with output_area:
        output_area.clear_output()
        if text_input.value and 'chain_instance' in globals():
            user_question = text_input.value
            try:
                response = chain_instance({"question": user_question})
                assistant_response = response['answer']
                print(f"Response: {assistant_response}")
            except Exception as e:
                print(f"Error querying the model: {e}")
        else:
            print("Model is not loaded or no question entered.")

query_button.on_click(on_query_button_clicked)

# **AI VOICE ASSISTANT**

In [12]:
!pip install --upgrade langchain langchain_community
!pip install gtts
!pip install groq
!pip install python-dotenv
!pip install ipywidgets

**This script uses the Groq API with the llama3-70b-8192 model to generate text responses based on user input. The gTTS library converts the generated text into speech, which is then played automatically. Widgets from ipywidgets create an interactive interface where users can input text, generate responses, and hear them spoken aloud.**

In [13]:
import os
from dotenv import load_dotenv
from gtts import gTTS
from groq import Groq
import base64
from IPython.display import display, HTML, Audio
import ipywidgets as widgets

from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')


def generate_response(user_input):
    client = Groq(api_key=groq_api_key)
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": user_input
            }
        ],
        model="llama3-70b-8192",
        temperature=0.5,
        max_tokens=1024,
        top_p=1
    )
    completion = response.choices[0].message.content
    return completion

def TTS(text):
    filename = "output.mp3"
    try:
        tts = gTTS(text=text, lang='en')
        tts.save(filename)
        return filename
    except Exception as e:
        print(f"Exception: {e}")
        return None

def autoplay_audio(file):
    display(Audio(file, autoplay=True))

def on_button_click(b):
    user_input = input_text.value
    response = generate_response(user_input)
    file = TTS(response)
    if file:
        autoplay_audio(file)

# Create widgets with improved styling
input_text = widgets.Textarea(
    value='Hello, how are you?',
    description='Input:',
    layout=widgets.Layout(width='80%', height='100px'),
    style={'description_width': 'initial'}
)

button = widgets.Button(
    description="Generate Response",
    button_style='success',
    layout=widgets.Layout(width='30%')
)
button.on_click(on_button_click)

output_area = widgets.Output()

# Display widgets
display(HTML("<h2>Interactive Chat and TTS Interface</h2>"), input_text, button, output_area)



Textarea(value='Hello, how are you?', description='Input:', layout=Layout(height='100px', width='80%'), style=…

Button(button_style='success', description='Generate Response', layout=Layout(width='30%'), style=ButtonStyle(…

Output()

# **MP4 OR MP3 OR MOV UPLOAD TO TEXT AND AUDIO**

This script sets up an interactive interface for processing audio files and generating responses based on user queries. It performs the following steps:

Audio Processing: Users upload audio files, which are split into chunks for transcription. Each chunk is then transcribed into text using the Groq API.
Text Processing and Search: The transcribed text is split into smaller chunks and indexed using FAISS. Users can then query this index to find relevant text segments, which are used to generate answers to their questions through the Groq chat completion API.
Interface and Output: The script uses widgets to manage file uploads, prompt inputs, and responses. The generated responses are converted to speech using Google Text-to-Speech (gTTS) and played back to the user.

In [14]:
!pip install faiss-cpu
!pip install gtts
!pip install ipywidgets
!pip install pandas
!pip install numpy
!pip install pydub
!pip install langchain
!pip install langchain_pinecone
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.2 MB/s eta 0:00:00


In [15]:
import os
import numpy as np
import faiss
from pydub import AudioSegment
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from gtts import gTTS
from IPython.display import display, Audio
import ipywidgets as widgets
from dotenv import load_dotenv
from groq import Groq
from io import BytesIO
from google.colab import userdata

# Load environment variables
load_dotenv()

# Retrieve Groq API key
groq_api_key = userdata.get('GROQ_API_KEY')

# Initialize Groq client
client = Groq(api_key=groq_api_key)

# Global variables
documents = []
faiss_index = None

# Audio to Text
def audio_to_text(audio_file):
    with audio_file:
        translation = client.audio.translations.create(
            file=(audio_file.name, audio_file.read()),
            model="whisper-large-v3",
        )
    return translation.text

# Transcript Chat Completion
def transcript_chat_completion(client, transcript, user_question):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f'Use this transcript or transcripts to answer any user questions, citing specific quotes:\n\n{transcript}'
            },
            {
                "role": "user",
                "content": user_question,
            }
        ],
        model="llama3-70b-8192",
    )
    return chat_completion.choices[0].message.content

# Splitting Audio into Chunks
def split_audio(audio_file, chunk_length_ms, overlap_ms, output_folder):
    audio = AudioSegment.from_file(audio_file, format="mp3")
    num_chunks = len(audio) // (chunk_length_ms - overlap_ms) + (1 if len(audio) % chunk_length_ms else 0)

    for i in range(num_chunks):
        start_ms = i * chunk_length_ms - (i * overlap_ms)
        end_ms = start_ms + chunk_length_ms
        chunk = audio[start_ms:end_ms]
        export_fp = os.path.join(output_folder, f"chunk_{i+1}.mp3")
        chunk.export(export_fp, format="mp3")

    return num_chunks

# Transcribe and Process Chunks
def process_chunks(chunk_folder, prompt_text):
    chunk_fps = [os.path.join(chunk_folder, fp) for fp in os.listdir(chunk_folder)]

    text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=20)
    global documents
    documents = []

    for filepath in chunk_fps:
        transcript = audio_to_text(open(filepath, 'rb'))
        chunks = text_splitter.split_text(transcript)
        for chunk in chunks:
            header = f"Prompt: {prompt_text}\n\n"
            documents.append(Document(page_content=header + chunk, metadata={"source": "local"}))

    return documents

# Initialize FAISS
def build_faiss_index(documents, embedding_model_name="all-MiniLM-L6-v2"):
    embedding_function = HuggingFaceEmbeddings(model_name=embedding_model_name)
    embeddings = np.array([embedding_function.embed_query(doc.page_content) for doc in documents])

    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)

    return index, embeddings

def search_faiss_index(index, query, k=3, embedding_model_name="all-MiniLM-L6-v2"):
    embedding_function = HuggingFaceEmbeddings(model_name=embedding_model_name)
    query_embedding = embedding_function.embed_query(query)
    query_embedding = np.array([query_embedding])

    distances, indices = index.search(query_embedding, k)
    return distances, indices

# Widgets for Interaction
def on_upload_change(change):
    if change['new']:
        for name, uploaded_file in upload_widget.value.items():
            file_path = os.path.join("uploads", name)
            with open(file_path, 'wb') as f:
                f.write(uploaded_file['content'])

            # Process the uploaded file
            split_audio(file_path, chunk_length_ms=1000000, overlap_ms=10000, output_folder="mp3-chunks")
            process_chunks("mp3-chunks", prompt_text.value)
            global faiss_index
            faiss_index, _ = build_faiss_index(documents)
            status_label.value = "File processed and FAISS index built successfully."

def on_button_click(b):
    user_question = input_text.value
    with output:
        output.clear_output()
        if faiss_index:
            relevant_docs = search_faiss_index(faiss_index, user_question)
            relevant_transcripts = '\n\n------------------------------------------------------\n\n'.join([documents[i].page_content for i in relevant_docs[1][0]])
            response = transcript_chat_completion(client, relevant_transcripts, user_question)
            print("**Answer:**", response)
            tts_file = TTS(response)
            if tts_file:
                display(Audio(tts_file, autoplay=True))
        else:
            print("Please upload and process a file first.")

# Text-to-Speech
def TTS(text):
    filename = "output.mp3"
    try:
        tts = gTTS(text=text, lang='en')
        tts.save(filename)
        return filename
    except Exception as e:
        print(f"Exception: {e}")
        return None

# Widgets
upload_widget = widgets.FileUpload(
    accept='.mp3,.mp4',  # Accept both audio and video files
    multiple=False,
    description='Upload File'
)
upload_widget.observe(on_upload_change, names='value')

prompt_text = widgets.Text(
    value='',
    description='Enter prompt:',
    disabled=False
)

input_text = widgets.Text(
    value='',
    description='Enter your question:',
    disabled=False
)

button = widgets.Button(description="Generate Response")
button.on_click(on_button_click)

output = widgets.Output()
status_label = widgets.Label(value="Upload a file and enter a prompt to start.")

# Display Widgets
display(widgets.VBox([upload_widget, prompt_text, input_text, button, status_label, output]))

# Ensure the uploads directory exists
if not os.path.exists("uploads"):
    os.makedirs("uploads")
if not os.path.exists("mp3-chunks"):
    os.makedirs("mp3-chunks")



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# ***YOUTUBE URL***

This script provides a complete workflow for processing YouTube videos into text transcripts. It downloads audio from a YouTube URL using yt-dlp, splits the audio into chunks, transcribes each chunk (mock implementation), and stores the combined transcript in a GROQ database. Users can input the URL, chunk size, and a query to interact with the stored transcript through an IPython widget interface.

In [16]:
!pip install yt-dlp
!pip install pydub
!pip install ipywidgets
!apt-get install ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [19]:
import os
import shutil
import logging
import traceback
import re
import time
from typing import List
import yt_dlp as youtube_dl  # Updated import for yt-dlp
from pydub import AudioSegment
from ipywidgets import widgets, interactive
from IPython.display import display
import groq

# Set up logging
logging.basicConfig(level=logging.INFO)

# Retrieve Groq API key
groq_api_key = userdata.get('GROQ_API_KEY')

# Initialize Groq client
client = Groq(api_key=groq_api_key)


# Define mock transcription function
def transcribe_audio(file_path: str) -> str:
    """
    Mock function to simulate transcription. Replace with actual transcription API call if needed.
    """
    return "This is a mock transcription of the audio file."

# Define function to download YouTube audio using yt-dlp
def youtubedl_download(yt_url: str, output_path: str = None) -> str:
    if output_path is None:
        output_path = os.getcwd()

    try:
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': os.path.join(output_path, 'audio.%(ext)s'),
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
        }

        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(yt_url, download=True)
            file_path = os.path.join(output_path, f"audio.{info_dict['ext']}")
            mp3_file_path = file_path.rsplit('.', 1)[0] + '.mp3'
            logging.info(f"Saved YouTube audio to: {mp3_file_path}")
            return mp3_file_path
    except Exception as e:
        logging.error(f"Download failed: {e}")
        logging.error(traceback.format_exc())
        raise

# Define function to create audio chunks
def create_audio_chunks(audio_file: str, chunk_size: int, temp_dir: str) -> List[str]:
    os.makedirs(temp_dir, exist_ok=True)
    file_name = os.path.splitext(os.path.basename(audio_file))[0]

    try:
        audio = AudioSegment.from_file(audio_file)
    except Exception as e:
        logging.error(f"Failed to load audio file: {e}")
        logging.error(traceback.format_exc())
        return []

    start = 0
    end = chunk_size
    counter = 0
    chunk_files = []

    while start < len(audio):
        chunk = audio[start:end]
        chunk_file_path = os.path.join(temp_dir, f"{counter}_{file_name}.mp3")
        try:
            chunk.export(chunk_file_path, format="mp3")
            chunk_files.append(chunk_file_path)
        except Exception as e:
            logging.error(f"Failed to export chunk {counter}: {e}")
            logging.error(traceback.format_exc())
            raise
        start += chunk_size
        end += chunk_size
        counter += 1
    return chunk_files

# Define function to generate transcript
def generate_youtube_transcript(yt_url: str, chunk_size: int, temp_dir: str) -> str:
    youtube_url_pattern = r'^(https?://)?(www\.)?(youtube\.com|youtu\.be)/.+$'
    if not re.match(youtube_url_pattern, yt_url):
        raise ValueError("Invalid YouTube URL provided.")

    try:
        file_path = youtubedl_download(yt_url)
    except Exception as e:
        logging.error(f"Failed to download YouTube video: {e}")
        logging.error(traceback.format_exc())
        raise

    try:
        chunk_files = create_audio_chunks(file_path, chunk_size, temp_dir)
    except Exception as e:
        logging.error(f"Failed to create audio chunks: {e}")
        logging.error(traceback.format_exc())
        raise

    transcripts = []
    for file_name in chunk_files:
        try:
            logging.info(f"Transcribing {file_name}")
            transcript = transcribe_audio(file_name)
            transcripts.append(transcript)
        except Exception as e:
            logging.error(f"Failed to transcribe file {file_name}: {e}")
            logging.error(traceback.format_exc())
            raise

    full_transcript = " ".join(transcripts)

    try:
        shutil.rmtree(temp_dir)
        os.remove(file_path)
    except Exception as e:
        logging.error(f"Failed to clean up files: {e}")
        logging.error(traceback.format_exc())
        raise

    return full_transcript

# Define function to store the transcript in GROQ
def store_transcript_in_groq(transcript: str, document_id: str):
    try:
        response = client.documents.create({
            'id': document_id,
            'transcript': transcript
        })
        return response
    except Exception as e:
        logging.error(f"Failed to store transcript in GROQ: {e}")
        logging.error(traceback.format_exc())
        raise

# Define function to query GROQ
def query_groq(document_id: str, query: str) -> str:
    try:
        response = client.documents.query({
            'id': document_id,
            'query': query
        })
        return response.get('result', 'No result found')
    except Exception as e:
        logging.error(f"Failed to query GROQ: {e}")
        logging.error(traceback.format_exc())
        raise

# Define function to handle user input and display results
def on_button_click(yt_url, chunk_size, query):
    temp_dir = "temp_chunks"
    document_id = "youtube_transcript_" + str(int(time.time()))  # Unique document ID
    try:
        logging.info("Generating transcript...")
        transcript = generate_youtube_transcript(yt_url, chunk_size, temp_dir)
        print("Transcript generated successfully.")
        print("Here's a snippet of the transcript:")
        print(transcript[:1000])  # Print the first 1000 characters

        # Store the transcript in GROQ
        store_transcript_in_groq(transcript, document_id)

        # Query the transcript
        answer = query_groq(document_id, query)
        print("\nAnswer to your query:")
        print(answer)

    except Exception as e:
        print(f"An error occurred: {e}")

# Create widgets
yt_url_widget = widgets.Text(
    description='YouTube URL:',
    placeholder='Enter YouTube URL here'
)

chunk_size_widget = widgets.IntText(
    description='Chunk Size (ms):',
    value=60000,  # Default 1 minute chunks
    min=1000,
    step=1000
)

query_widget = widgets.Text(
    description='Query:',
    placeholder='Enter your query here'
)

button = widgets.Button(description="Generate Transcript & Query")
button.on_click(lambda b: on_button_click(yt_url_widget.value, chunk_size_widget.value, query_widget.value))

display(yt_url_widget, chunk_size_widget, query_widget, button)



Text(value='', description='YouTube URL:', placeholder='Enter YouTube URL here')

IntText(value=60000, description='Chunk Size (ms):', step=1000)

Text(value='', description='Query:', placeholder='Enter your query here')

Button(description='Generate Transcript & Query', style=ButtonStyle())

# **ML Assistant**

This script provides an interactive interface using IPython widgets to facilitate the creation of a machine learning project. Users select a model, upload a dataset, and describe their problem. The script then uses predefined agents to clarify the problem, assess the data, recommend models, and generate starter code, orchestrating these tasks through the Crew framework and displaying the results.

In [20]:
!pip install crewai
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 16.9 MB/s eta 0:00:00
 

In [23]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from io import StringIO
from crewai import Agent, Task, Crew
from langchain_groq import ChatGroq

# Retrieve Groq API key
api_key = userdata.get('GROQ_API_KEY')

def setup_llm(model_name):
    return ChatGroq(
        temperature=0,
        groq_api_key=api_key,  # Ensure the API key is assigned correctly
        model_name=model_name
    )

def create_agents(llm):
    Problem_Definition_Agent = Agent(
        role='Problem_Definition_Agent',
        goal="""Clarify the machine learning problem the user wants to solve,
            identifying the type of problem (e.g., classification, regression) and any specific requirements.""",
        backstory="""You are an expert in understanding and defining machine learning problems.
            Your goal is to extract a clear, concise problem statement from the user's input,
            ensuring the project starts with a solid foundation.""",
        verbose=True,
        allow_delegation=False,
        llm=llm,
    )

    Data_Assessment_Agent = Agent(
        role='Data_Assessment_Agent',
        goal="""Evaluate the data provided by the user, assessing its quality,
            suitability for the problem, and suggesting preprocessing steps if necessary.""",
        backstory="""You specialize in data evaluation and preprocessing.
            Your task is to guide the user in preparing their dataset for the machine learning model,
            including suggestions for data cleaning and augmentation.""",
        verbose=True,
        allow_delegation=False,
        llm=llm,
    )

    Model_Recommendation_Agent = Agent(
        role='Model_Recommendation_Agent',
        goal="""Suggest the most suitable machine learning models based on the problem definition
            and data assessment, providing reasons for each recommendation.""",
        backstory="""As an expert in machine learning algorithms, you recommend models that best fit
            the user's problem and data. You provide insights into why certain models may be more effective than others,
            considering classification vs regression and supervised vs unsupervised frameworks.""",
        verbose=True,
        allow_delegation=False,
        llm=llm,
    )

    Starter_Code_Generator_Agent = Agent(
        role='Starter_Code_Generator_Agent',
        goal="""Generate starter Python code for the project, including data loading,
            model definition, and a basic training loop, based on findings from the problem definitions,
            data assessment and model recommendation""",
        backstory="""You are a code wizard, able to generate starter code templates that users
            can customize for their projects. Your goal is to give users a head start in their coding efforts.""",
        verbose=True,
        allow_delegation=False,
        llm=llm,
    )

    return Problem_Definition_Agent, Data_Assessment_Agent, Model_Recommendation_Agent, Starter_Code_Generator_Agent

def on_submit(b):
    clear_output(wait=True)
    display(model_selector)
    display(upload_widget)
    display(problem_description)
    display(submit_button)

    model_name = model_selector.value
    llm = setup_llm(model_name)
    Problem_Definition_Agent, Data_Assessment_Agent, Model_Recommendation_Agent, Starter_Code_Generator_Agent = create_agents(llm)

    problem_text = problem_description.value
    data_file = upload_widget.value

    # Create tasks
    task_define_problem = Task(
        description="""Clarify and define the machine learning problem,
                       including identifying the problem type and specific requirements.
                       Here is the user's problem: {ml_problem}""".format(ml_problem=problem_text),
        agent=Problem_Definition_Agent,
        expected_output="A clear and concise definition of the machine learning problem."
    )

    if data_file:
        # Extract the file content from the uploaded file
        file_content = list(data_file.values())[0]['content']
        df = pd.read_csv(StringIO(file_content.decode('utf-8')))
        df_sample = df.head(5)
        task_assess_data = Task(
            description="""Evaluate the user's data for quality and suitability,
                           suggesting preprocessing or augmentation steps if needed.
                           Here is a sample of the user's data:
                           {df_sample}""".format(df_sample=df_sample.to_csv(index=False)),
            agent=Data_Assessment_Agent,
            expected_output="An assessment of the data's quality and suitability, with suggestions for preprocessing or augmentation if necessary."
        )
    else:
        task_assess_data = Task(
            description="""The user has not uploaded any specific data for this problem,
                           but please go ahead and consider a hypothetical dataset that might be useful
                           for their machine learning problem.""",
            agent=Data_Assessment_Agent,
            expected_output="A hypothetical dataset that might be useful for the user's machine learning problem, along with any necessary preprocessing steps."
        )

    task_recommend_model = Task(
        description="""Suggest suitable machine learning models for the defined problem
                       and assessed data, providing rationale for each suggestion.""",
        agent=Model_Recommendation_Agent,
        expected_output="A list of suitable machine learning models for the defined problem and assessed data, along with the rationale for each suggestion."
    )

    task_generate_code = Task(
        description="""Generate starter Python code tailored to the user's project using the model recommendation agent's recommendation(s),
                       including snippets for package import, data handling, model definition, and training""",
        agent=Starter_Code_Generator_Agent,
        expected_output="Python code snippets for package import, data handling, model definition, and training, tailored to the user's project, plus a brief summary of the problem and model recommendations."
    )

    crew = Crew(
        agents=[Problem_Definition_Agent, Data_Assessment_Agent, Model_Recommendation_Agent, Starter_Code_Generator_Agent],
        tasks=[task_define_problem, task_assess_data, task_recommend_model, task_generate_code],
        verbose=True  # Set verbose to a boolean value
    )

    result = crew.kickoff()
    print(result)


# Define widgets
model_selector = widgets.Dropdown(
    options=['llama3-8b-8192', 'mixtral-8x7b-32768', 'gemma-7b-it'],
    value='llama3-8b-8192',
    description='Choose a model:',
)

upload_widget = widgets.FileUpload(
    accept='.csv',  # Accept only CSV files
    multiple=False
)

problem_description = widgets.Textarea(
    description='Describe your ML problem:',
    placeholder='Enter the problem description here...',
    layout=widgets.Layout(width='100%', height='150px')
)

submit_button = widgets.Button(
    description='Submit',
    button_style='success'
)

submit_button.on_click(on_submit)

# Display widgets
display(model_selector)
display(upload_widget)
display(problem_description)
display(submit_button)


Dropdown(description='Choose a model:', options=('llama3-8b-8192', 'mixtral-8x7b-32768', 'gemma-7b-it'), value…

FileUpload(value={'AirPassengers-2.csv': {'metadata': {'name': 'AirPassengers-2.csv', 'type': 'text/csv', 'siz…

Textarea(value='linear regression model', description='Describe your ML problem:', layout=Layout(height='150px…

Button(button_style='success', description='Submit', style=ButtonStyle())

 [2024-09-12 12:37:45][DEBUG]: == Working Agent: Problem_Definition_Agent
 [2024-09-12 12:37:45][INFO]: == Starting Task: Clarify and define the machine learning problem,
                       including identifying the problem type and specific requirements.
                       Here is the user's problem: linear regression model


> Entering new CrewAgentExecutor chain...
Thought: I now understand that the user has mentioned a linear regression model, which is a type of supervised machine learning problem. My goal is to clarify and define the machine learning problem, including identifying the problem type and specific requirements. I will ask follow-up questions to gather more information and provide a clear and concise definition of the machine learning problem.

Final Answer:

**Problem Definition:**

The machine learning problem is to develop a linear regression model that predicts a continuous output variable based on a set of input features. The goal is to create a model that

# **AUDIO LANGUAGE TRANSLATOR**

This code sets up an interactive audio translation interface using IPython widgets. Users can upload a WAV audio file, which is then transcribed to text using speech recognition. The transcribed text is subsequently translated into the selected language using the Groq API, with the translated text displayed in the output area.

In [24]:
!pip install ipywidgets groq pydantic pydub
!pip install SpeechRecognition


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 33.6 MB/s eta 0:00:00


In [26]:
import os
import ipywidgets as widgets
from IPython.display import display, Audio
import json
from typing import Optional
from groq import Groq
from pydantic import BaseModel
import io
from pydub import AudioSegment
import speech_recognition as sr

# Initialize Groq client
api_key = userdata.get('GROQ_API_KEY')
# Replace with your actual API key
client = Groq(api_key=api_key)

# Model for the translation
class Translation(BaseModel):
    text: str
    comments: Optional[str] = None

# Translate text using the Groq API
def groq_translate(query, from_language, to_language):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"You are a helpful assistant that translates text from {from_language} to {to_language}."
                           f"You will only reply with the translation text and nothing else in JSON."
                           f" The JSON object must use the schema: {json.dumps(Translation.model_json_schema(), indent=2)}",
            },
            {
                "role": "user",
                "content": f"Translate '{query}' from {from_language} to {to_language}."
            }
        ],
        model="mixtral-8x7b-32768",
        temperature=0.2,
        max_tokens=1024,
        stream=False,
        response_format={"type": "json_object"},
    )
    return Translation.model_validate_json(chat_completion.choices[0].message.content)

# Upload audio file function
def upload_audio(file):
    audio = AudioSegment.from_file(file, format="wav")
    with open("uploaded_audio.wav", "wb") as out_f:
        out_f.write(audio.export(format="wav").read())
    return "uploaded_audio.wav"

# Convert audio to text using speech_recognition
def audio_to_text(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            return text
        except sr.UnknownValueError:
            return "Audio could not be understood"
        except sr.RequestError:
            return "Could not request results from Google Speech Recognition service"

# Language options with Hindi added
languages = {
   "Portuguese": "pt",
   "Spanish": "es",
   "German": "de",
   "French": "fr",
   "Italian": "it",
   "Dutch": "nl",
   "Russian": "ru",
   "Japanese": "ja",
   "Chinese": "zh",
   "Korean": "ko",
   "Hindi": "hi"
}

# Reverse the language dictionary for lookup
reverse_languages = {v: k for k, v in languages.items()}

# Widgets for the interface
language_dropdown = widgets.Dropdown(
    options=[(name, code) for name, code in languages.items()],
    description='Translate to:',
    value=None,
    disabled=False,
    layout=widgets.Layout(width='50%')
)

upload_button = widgets.FileUpload(
    accept='.wav',  # Accept only .wav files
    multiple=False,  # Do not allow multiple files
    description='Upload Audio',
    layout=widgets.Layout(width='50%')
)

translate_button = widgets.Button(
    description="Translate",
    disabled=False,
    button_style='primary',
    tooltip='Click to translate text',
    icon='language',
    layout=widgets.Layout(width='50%')
)

output_area = widgets.Output()

def on_upload_change(change):
    with output_area:
        output_area.clear_output()
        if upload_button.value:
            file = list(upload_button.value.values())[0]
            audio_file = upload_audio(io.BytesIO(file['content']))
            display(Audio(audio_file, autoplay=True))

def on_translate_button_clicked(b):
    with output_area:
        output_area.clear_output()
        to_language_code = language_dropdown.value
        to_language = reverse_languages.get(to_language_code)
        if to_language:
            audio_file = "uploaded_audio.wav"
            transcribed_text = audio_to_text(audio_file)
            if transcribed_text:
                translated = groq_translate(transcribed_text, 'en', to_language)
                print(f"Translated Text: {translated.text}")

# Bind actions to buttons
upload_button.observe(on_upload_change, names='value')
translate_button.on_click(on_translate_button_clicked)

# Display widgets in a vertical box layout
widget_box = widgets.VBox([
    widgets.HTML(value="<h2>Audio Translation Interface</h2>"),
    language_dropdown,
    upload_button,
    translate_button,
    output_area
])

display(widget_box)



